In [1]:
import sys
from terminaltables import AsciiTable

# Distance-vector routing protocol
A distance-vector routing protocol in data networks determines the best route for data packets based on distance. To establish the best route, routers regularly exchange information with neighboring routers, usually their routing table, hop count for a destination network and possibly other traffic related information. Routers that implement distance-vector protocol rely purely on the information provided to them by other routers, and do not assess the network topology. Distance-vector protocols update the routing tables of routers and determine the route on which a packet will be sent by the next hop which is the exit interface of the router and the IP address of the interface of the receiving router. Distance is a measure of the cost to reach a certain node. The least cost route between any two nodes is the route with minimum distance. 


## 1. Bellman–Ford algorithm
The Bellman–Ford algorithm is an algorithm that computes shortest paths from a single source vertex to all of the other vertices in a weighted digraph. It is slower than Dijkstra's algorithm for the same problem, but more versatile, as it is capable of handling graphs in which some of the edge weights are negative numbers. The algorithm was first proposed by Alfonso Shimbel in 1955, but is instead named after Richard Bellman and Lester Ford, Jr., who published it in 1958 and 1956, respectively. Edward F. Moore also published the same algorithm in 1957, and for this reason it is also sometimes called the Bellman–Ford–Moore algorithm.

### Notations
- $D_x(y)$: Distance of node $x$ to node $y$
- $c(x,y)$: Cost of the link $(x,y)$

#### Bellman-Ford Equation
For a source node $x$, and destination node $y$, given a set of neighbors $v \in N$, $D_x(y)$ is calculated as follows:
$ D_x(y) = min_v(c(x,v) + D_v(y))$

**Exercise:** Implement `bell_ford_eq()` to calculate Bellman-Ford equation. `bell_ford_eq()` argument is `neighbor_info` which is a Python dictionary with the following keys and values:
    
   1. `neighbor_info["distance"]`: is a dictionary with each key represents the source and its neighbor ids and each value is python dictionary where: 
    
      1. keys represent the node_ids that are discovered so far
    
      2. values represent the distance of the neighbor to that node.
    
   2. `neighbor_info["link_cost"]`: is a dictionary with keys as the neighbor ids and values that represent the corresponding link cost.




In [2]:
def bell_ford_eq(source_id, neighbor_info):
    
    # create an empty dictionary representing the distance vector
    # keys: represet the nodes in the network discovered so far
    # values: represent the distances to those nodes
    distance_vector = {}
    
    
    # initialization
    
    # set the distance to the source_id as zero
    distance_vector[source_id] = 0
    
    # set the initial distance of the neighbors as the corresponding link cost
    for neighbor_id in neighbor_info['link_cost']:
        # set the distance to link cost
        distance_vector[neighbor_id] = neighbor_info['link_cost'][neighbor_id]
        

    # step 1: gather the node ids that are discovered so far

    # add other node ids of other nodes that are discovered by neighbor_id to the keys 
    for neighbor_id in neighbor_info['distance']:
        for node_id in neighbor_info['distance'][neighbor_id]:
            # if node_id already exist, skip it
            if node_id in distance_vector:
                continue
                
            # set the distance to infinity: sys.maxsize 
            distance_vector[node_id] = sys.maxsize 


    # step 2: calculate the distance using bellman-ford to the nodes discovered so far
    for node_id in distance_vector:
        # if node_id is source_id, then skip it
        if node_id is source_id:
            continue
        for neighbor_id in neighbor_info['distance']:
            # if node_id is not in the neighbor_id distance vector, skip this neighbor
            if node_id not in neighbor_info['distance'][neighbor_id]:
                continue

            # calculate Bellman-ford
            distance = neighbor_info['link_cost'][neighbor_id] + neighbor_info['distance'][neighbor_id][node_id]

            # update the distance vector if distance is less than the current one
            if distance < distance_vector[node_id]:
                distance_vector[node_id] = distance
                
                
    
    return distance_vector



In [3]:
neighbor_info = {'distance': {'v': {'z': 5},
                              'x': {'z': 3},
                              'w': {'z': 3}
                             },
                 'link_cost': {'u': 0,
                               'v': 2,
                               'x': 1,
                               'w': 5}
                }

source_id = 'u'
bell_ford_eq(source_id, neighbor_info)


{'u': 0, 'v': 2, 'x': 1, 'w': 5, 'z': 4}

**Expected Output:** `{'u': 0, 'v': 2, 'x': 1, 'w': 5, 'z': 4}`

## 2. Utility functions

### 2.1 Get neighbors and its costs
**Exercise:** Implement `get_neighbors_link_cost()` that given a network topology, for a given `source_id`, returns a dictionary with keys representing `source_id` neighbors' ids and values that represent the corresponding link cost from the `source_id` to each neighbor. Assume that `network_topology` is a Python dictionary with the following keys and values:

- `network_topology["nodes"]`: is a set of numbers representing the node ids (router ids, e.g., 1,2,3) in the network.
- `network_topology["links"]`: is a dictionary with keys as link tuples (e.g., (1,2)) and values that represent the corresponding link cost.


In [12]:
def get_neighbors_link_cost(source_id, network_topology):
    # get network topology nodes
    nodes = network_topology['nodes']
    
    # get network topology links
    links = network_topology['links'].keys()
    
    # get the link costs in network topology
    link_cost = network_topology['links']

    # make an empty dictionary for neighbor_link_cost
    neighbor_link_cost = {}
    
    # add the cost to the source_id as zero
    neighbor_link_cost[source_id] = 0
    
    # go through the link
    for link in links:
        # if source id is not in the link, skip it
        if source_id not in link:
            continue
        # get the neighbor information and cost for this neighbor
        neighbor = link[0] if source_id == link[1] else link[1]
        neighbor_link_cost[neighbor] = link_cost[link]
    
    return neighbor_link_cost
    

In [13]:
network_top = {'nodes': set(['z','x','y','w','v','u','t']),
              'links': {('z','x'): 8,
                        ('z','y'): 12,
                        ('x','y'): 6,
                        ('x','w'): 6,
                        ('x','v'): 3,
                        ('y','t'): 7,
                        ('y','v'): 8,
                        ('v','t'): 4,
                        ('v','w'): 4,
                        ('v','u'): 3,
                        ('t','u'): 2,
                        ('w','u'): 3
                       }
              }
source_id = 'v'
get_neighbors_link_cost(source_id, network_top)

{'v': 0, 'x': 3, 'y': 8, 't': 4, 'w': 4, 'u': 3}

**Expected output:** `{'v': 0, 'x': 3, 'y': 8, 't': 4, 'w': 4, 'u': 3}`

### 2.2 Display Node's distance vector table
**Exercise:** Implement `display_DV_table()` to show a given distance vector table similar to the table discussed in class. Remember, for each source_id, each row represents the node's neighbor (including the source_id) and each column represents the nodes that are discovered in the network (including the source_id). Each value in the table is  a distance. Assume that the input to the `display_DV_table()` is a dictionary with each key representing the source and its neighbor ids and each value is python dictionary where: 
    
  1. keys represent the node_ids that are discovered so far
    
  2. values represent the distance of the neighbor to that node.

In [14]:
def display_DV_table(source_id, neighbor_distance_info):
    
    DV_table = []
    
    columns = ['Node: ' + str(source_id) , str(source_id)]
    # add other node ids of other nodes that are discovered by neighbor_id to the keys 
    for neighbor_id in neighbor_distance_info:
        for node_id in neighbor_distance_info[neighbor_id]:
            # if node_id already exist, skip it
            if node_id in columns:
                continue
            # append node_id to colums
            columns.append(node_id)
            
    # append columns to the DV_table
    DV_table.append(columns)
    
    for neighbor_id in neighbor_distance_info:
        row = [str(neighbor_id)]
        for col in columns[1:]:
            for node_id in neighbor_distance_info[neighbor_id]:
                # if node_id already exist, skip it
                if node_id is not col:
                    continue
                row.append(str(neighbor_distance_info[neighbor_id][node_id]))
        DV_table.append(row)
    
    table = AsciiTable(DV_table)
    print(table.table)

In [15]:
neighbor_distance_info = {'w': {'w': 0, 'x': 6, 'v': 4, 'u': 3}}
source_id = 'w'
display_DV_table(source_id, neighbor_distance_info)
                          

+---------+---+---+---+---+
| Node: w | w | x | v | u |
+---------+---+---+---+---+
| w       | 0 | 6 | 4 | 3 |
+---------+---+---+---+---+


**Expected output:** 

```
+---------+---+---+---+---+
| Node: w | w | x | v | u |
+---------+---+---+---+---+
| w       | 0 | 6 | 4 | 3 |
+---------+---+---+---+---+
```

## 3. Distance Vector Simulator

Distance vector updates are performed periodically in a distance-vector protocol where all or part of a router's routing table is sent to all its neighbors that are configured to use the same distance-vector routing protocol. Once a router has this information it is able to amend its own routing table to reflect the changes and then inform its neighbors of the changes. This process has been described as *routing* by *rumor* because routers are relying on the information they receive from other routers and cannot determine if the information is actually valid and true. There are a number of features which can be used to help with instability and inaccurate routing information. 

**Exercise:** Implement `distance_vector_simulator()` that simulate the distance vector updates throughout the network.

In [26]:
def distance_vector_simulator(network_topology):
    # get network topology nodes
    nodes = network_topology['nodes']
    
    # get network topology links
    links = network_topology['links'].keys()
    
    # get the link costs in network topology
    link_cost = network_topology['links']
    
    ### initialization
    
    # set step to be zero
    step = 0
    
    print('*************** step: {0} **************'.format(str(step)))
    
    # make an empty dictionary for neighbor info
    neighbor_info = {}
    
    # for each node
    for source_id in nodes:
        # make an empty neighbor_info dictionary (recall neighbor_info format!)
        neighbor_info[source_id] = {'distance': {},
                              'link_cost': {}
                              }
        # find the neighbors and their costs for this node 
        # update the "link_cost" of neighbor_info with link_cost information
        neighbor_info[source_id]['link_cost'] = get_neighbors_link_cost(source_id, network_topology)
        
        # calculate the initial distance vector for this node
        # Note: this represents step 0 before any information exchange among neighbors
        # update the "distance" of the neighbor info
        neighbor_info[source_id]['distance'][source_id] = bell_ford_eq(source_id, neighbor_info[source_id])
        
        # display the initial DV table before any DV updates
        display_DV_table(source_id, neighbor_distance_info=neighbor_info[source_id]['distance'])
    
    # initially all nodes send their distance vector information to their neighbors
    # so all nodes are transmitters
    transmitters = []
    for node in nodes:
        # append node to transmitters
        transmitters.append(node)

    # loop until there is no update left in the network
    
    while transmitters:
        step += 1
        print('*************** step: {0} **************'.format(str(step)))
        print('Transmitters: ', transmitters)
        for source_id in nodes:
            # get the source_id neighbors
            source_id_neighbors = neighbor_info[source_id]['link_cost'].keys()
            
            # each node receives the distance vector from its neighbors,
            # if its neighbor is in "transmitters" list
            for neighbor_id in source_id_neighbors:
                # if the neighbor is not in the "transmitters" list, then skip it
                if neighbor_id not in transmitters:
                    continue
                
                # add neighbor_id distance vector information to source_id's neighbor_info database
                neighbor_info[neighbor_id]['distance'][source_id] = neighbor_info[source_id]['distance'][source_id]
            
            
        for source_id in nodes:
            # after receiveing distance vector from all neighbors
            # update the distances using bellman-ford
            new_distance_vector = bell_ford_eq(source_id, neighbor_info[source_id])
            
            # compare the new distance vector with the current one to see if there is an update in nodes/distances
            is_update = new_distance_vector != neighbor_info[source_id]['distance'][source_id]
            
            # if there is an update, then update the distance vector
            if is_update:
                neighbor_info[source_id]['distance'][source_id] = new_distance_vector
            # if there is no update
            else:
                # remove source_id from the "transmitters" list
                if source_id in transmitters:
                    # use remove() to remove the source_id from "transmitters"
                    # Note: this is the stopping criteria for the loop so make sure to remove the node!
                    transmitters.remove(source_id)
                
            # display the distance vector for the source_id after each step
            display_DV_table(source_id, neighbor_distance_info=neighbor_info[source_id]['distance']) 
               
            
    return neighbor_info
    

In [27]:
network_topology = {}
network_topology['nodes'] = set(['x','y','z'])
network_topology['links'] = {('x','y'): 2, ('y','z'): 1,  ('x','z'): 7}
network_distance_vectors = distance_vector_simulator(network_topology)

*************** step: 0 **************
+---------+---+---+---+
| Node: z | z | y | x |
+---------+---+---+---+
| z       | 0 | 1 | 7 |
+---------+---+---+---+
+---------+---+---+---+
| Node: y | y | x | z |
+---------+---+---+---+
| y       | 0 | 2 | 1 |
+---------+---+---+---+
+---------+---+---+---+
| Node: x | x | y | z |
+---------+---+---+---+
| x       | 0 | 2 | 7 |
+---------+---+---+---+
*************** step: 1 **************
Transmitters:  ['z', 'y', 'x']
+---------+---+---+---+
| Node: z | z | y | x |
+---------+---+---+---+
| z       | 0 | 1 | 3 |
| y       | 1 | 0 | 2 |
| x       | 7 | 2 | 0 |
+---------+---+---+---+
+---------+---+---+---+
| Node: y | y | x | z |
+---------+---+---+---+
| y       | 0 | 2 | 1 |
| z       | 1 | 7 | 0 |
| x       | 2 | 0 | 7 |
+---------+---+---+---+
+---------+---+---+---+
| Node: x | x | y | z |
+---------+---+---+---+
| x       | 0 | 2 | 3 |
| z       | 7 | 1 | 0 |
| y       | 2 | 0 | 1 |
+---------+---+---+---+
*************** step: 2 ***

**Expected output:** Make sure the generated tables matche with the ones in the figure 5.6 (page 387) of the textbook.



In [28]:
network_top = {'nodes': set(['z','x','y','w','v','u','t']),
              'links': {('z','x'): 8,
                        ('z','y'): 12,
                        ('x','y'): 6,
                        ('x','w'): 6,
                        ('x','v'): 3,
                        ('y','t'): 7,
                        ('y','v'): 8,
                        ('v','t'): 4,
                        ('v','w'): 4,
                        ('v','u'): 3,
                        ('t','u'): 2,
                        ('w','u'): 3
                       }
              }
network_distance_vectors = distance_vector_simulator(network_top)

*************** step: 0 **************
+---------+---+---+----+
| Node: z | z | x | y  |
+---------+---+---+----+
| z       | 0 | 8 | 12 |
+---------+---+---+----+
+---------+---+---+---+---+---+
| Node: x | x | z | y | w | v |
+---------+---+---+---+---+---+
| x       | 0 | 8 | 6 | 6 | 3 |
+---------+---+---+---+---+---+
+---------+---+---+---+---+---+---+
| Node: v | v | x | y | t | w | u |
+---------+---+---+---+---+---+---+
| v       | 0 | 3 | 8 | 4 | 4 | 3 |
+---------+---+---+---+---+---+---+
+---------+---+----+---+---+---+
| Node: y | y | z  | x | t | v |
+---------+---+----+---+---+---+
| y       | 0 | 12 | 6 | 7 | 8 |
+---------+---+----+---+---+---+
+---------+---+---+---+---+
| Node: w | w | x | v | u |
+---------+---+---+---+---+
| w       | 0 | 6 | 4 | 3 |
+---------+---+---+---+---+
+---------+---+---+---+---+
| Node: u | u | v | t | w |
+---------+---+---+---+---+
| u       | 0 | 3 | 2 | 3 |
+---------+---+---+---+---+
+---------+---+---+---+---+
| Node: t | t | y | v |

**Expected output:** compare the generated distance vector tables (values) for each node with the path cost you calculated using Dijkstra algorithm for problems P3 and P4 of Chapter 5 (page 429) of the textbook. These value should be identical. These values should also match the Dijsktra algorithm implementation you completed in the previous assignment.